In [6]:
import numpy as np
from scipy.stats import norm

## (1)

In [57]:
def MC(s0:float, k:float, r:float, q:float, sigma:float, T:float, 
       timestep:int, epoch:int, call_option:bool, seednum:int) -> float:
    '''
    Implementation of monte-carlo simulation, assuming each increment/step
    (change of price) is normally distributed, the drift and variance rate of
    x remain constant, equal to their values at time t, and in the time interval 
    of delta t.
    
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk free rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param timestep: number of steps within each time span
    @param epoch: number of paths
    @param call_option: if a call or put option
    @param seednum: random seed

    @return out: list of simulated price of dimension epoch
    '''
    np.random.seed(seednum)

    mu = r - q
    var = sigma ** 2
    dt = T / timestep
    out = []

    # epoch start
    for e in range(epoch):

        # start simulation of each path
        dx = 0
        for _ in range(timestep):
            
            # start each dt
            dx += (mu - var/2)*dt + sigma*np.random.normal()*np.sqrt(dt)
        
        # add to the log of s0
        st = np.log(s0) + dx
        st = np.exp(st)
        
        # calculate the value of the option regarding st
        out.append(max(st-k, 0)) if call_option else out.append(max(k-st, 0))
    
    return np.mean(out)*np.exp(-r*T)

In [17]:
def BSM(s0:float, k:float, r:float, q:float, sigma:float, T:float,
        call_option:bool) -> float:
    '''
    Implementation of black shore.
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk free rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param call_option: if a call or put option
    @param seednum: random seed 
    
    @return out: value of the option 
    '''
    d1 = (np.log(s0/k) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    
    if call_option:
        out = s0 * np.exp(-q*T) * norm.cdf(d1) - k * np.exp(-r*T) * norm.cdf(d2)
    else:
        out = k * np.exp(-r*T) * norm.cdf(-d2) - s0 * np.exp(-q*T) * norm.cdf(-d1)
    return out

In [58]:
s0 = 100
k = 100 
r = 0.06
q = 0.06
sigma = 0.35 
T = 1 
timestep = 100 
epoch = 4000
call_option = True
seednum = 102623

plain_mc = MC(s0, k, r, q, sigma, T, timestep, epoch, call_option, seednum)
bsm = BSM(s0, k, r, q, sigma, T, call_option)
plain_mc, bsm

(13.502692972236593, 13.083027408250246)

## (2)

In [59]:
def Antithetic_MC(s0:float, k:float, r:float, q:float, sigma:float, T:float, 
       timestep:int, epoch:int, call_option:bool, seednum:int) -> float:
    '''
    Implementation of monte-carlo simulation using antithetic method.
    
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk-free interest rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param timestep: number of steps within each time span
    @param epoch: number of paths
    @param call_option: if a call or put option
    @param seednum: random seed

    @return out: estimated option price using Monte Carlo simulation with antithetic method
    '''
    np.random.seed(seednum)
    mu = r - q
    var = sigma ** 2
    dt = T / timestep
    out = []
   
    # epoch start
    for e in range(epoch):

        # start simulation of each path
        f1, f2 = 0, 0
        for _ in range(timestep):
            
            # start each dt
            z = np.random.normal()
            f1 += (mu - var/2)*dt + sigma*z*np.sqrt(dt)
            f2 += (mu - var/2)*dt + sigma*(-z)*np.sqrt(dt)
        
        # add to the log of s0
        st1 = np.exp(np.log(s0) + f1)
        st2 = np.exp(np.log(s0) + f2)
        op1 = max(st1-k, 0) if call_option else max(k-st1, 0)
        op2 = max(st2-k, 0) if call_option else max(k-st2, 0)
        
        # calculate the value of the option regarding st
        out.append((op1+op2)/2)
    
    return np.mean(out)*np.exp(-r*T)

In [72]:
ant_mc = Antithetic_MC(s0, k, r, q, sigma, T, timestep, epoch, call_option, seednum)
ant_mc, bsm

(13.34634759851727, 13.083027408250246)

## (3)

In [69]:
def Control_Variate_MC(s0:float, k:float, r:float, q:float, sigma:float, T:float, 
                       timestep:int, epoch:int, call_option:bool, seednum:int) -> float:
    '''
    Implementation of monte-carlo simulation using control variate method.
    
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk-free interest rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param timestep: number of steps within each time span
    @param epoch: number of paths
    @param call_option: if a call or put option
    @param seednum: random seed

    @return out: estimated option price using Monte Carlo simulation with control variate method
    '''
    np.random.seed(seednum)
    mu = r - q
    var = sigma ** 2
    dt = T / timestep
    f_A_hat, f_B_hat = [], []

    # epoch start
    for e in range(epoch):

        # start simulation of each path
        dx = 0
        for _ in range(timestep):
            
            # start each dt
            dx += (mu - var/2)*dt + sigma*np.random.normal()*np.sqrt(dt)
        
        # add to the log of s0
        st = np.log(s0) + dx
        st = np.exp(st)
        
        # calculate the value of the option regarding st
        f_A_hat.append(max(st-k, 0)) if call_option else f_A_hat.append(max(k-st, 0))
        f_B_hat.append(st)
    
    f_A_hat_num = np.mean(f_A_hat)
    # use 1/(n-1) for both covariance and variance due to sample
    # this is already the case for np.cov and var
    beta = (np.cov(f_A_hat, f_B_hat)/np.var(f_B_hat))[0, 1]
    f_B_hat_num = np.mean(f_B_hat)
    f_B = s0 * np.exp((r - q) * T)

    
    return (f_A_hat_num-beta*(f_B_hat_num-f_B))*np.exp(-r * T)

In [71]:
cv_mc = Control_Variate_MC(s0, k, r, q, sigma, T, timestep, epoch, call_option, seednum)
cv_mc, bsm

(13.217164428481622, 13.083027408250246)